# English_to_French_Translation_Using_Encoder_and_Decoder with LSTM

### Sequence To Sequence Learning in Keras blog link:- https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html


### Sequence To Sequence Learning Research Paper Link:- https://arxiv.org/abs/1409.3215

# Downloading the Dataset

In [1]:
!wget http://www.manythings.org/anki/fra-eng.zip

--2024-01-17 08:35:02--  http://www.manythings.org/anki/fra-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7833145 (7.5M) [application/zip]
Saving to: ‘fra-eng.zip’

fra-eng.zip         100%[===================>]   7.47M  14.6MB/s    in 0.5s    

2024-01-17 08:35:13 (14.6 MB/s) - ‘fra-eng.zip’ saved [7833145/7833145]



# Unzip the Dataset

In [2]:
!unzip -q fra-eng.zip -d /content/french-eng

In [3]:
# import libraries

from __future__ import print_function
from keras.models import Model
from keras.layers import Input,LSTM,Dense
import numpy as np

In [8]:
batch_s = 64 # Batch Size For Training
epochs = 100 #Nuber of Epochs to Train For
latent_dimension = 256 # Latent Dimensionality of the Encoding Space
number_samples = 10000 # Number of Samples to train on
data_path = "/content/french-eng/fra.txt" # Path to the Data txt File on Disk

In [9]:
# Vectrize the Data
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path,'r',encoding='utf-8') as f:
  lines = f.read().split("\n")

for line in lines[: min(number_samples,len(lines) - 1)]:
  input_text,target_text, _ =line.split('\t')
  # We Use 'tab' as the "start sequence" character
  # For the Targets and "\n" as "end sequence" character
  target_text = '\t' + target_text + '\n'
  input_texts.append(input_text)
  target_texts.append(target_text)
  for char in input_text:
    if char not in input_characters:
      input_characters.add(char)

  for char in target_text:
    if char not in target_characters:
      target_characters.add(char)

In [10]:
print("Number of Input Samples: ",len(input_texts))
print("Number of Target Samples: ",len(target_texts))

Number of Input Samples:  10000
Number of Target Samples:  10000


In [11]:
target_texts

['\tVa !\n',
 '\tMarche.\n',
 '\tEn route !\n',
 '\tBouge !\n',
 '\tSalut !\n',
 '\tSalut.\n',
 '\tCours\u202f!\n',
 '\tCourez\u202f!\n',
 '\tPrenez vos jambes à vos cous !\n',
 '\tFile !\n',
 '\tFilez !\n',
 '\tCours !\n',
 '\tFuyez !\n',
 '\tFuyons !\n',
 '\tCours\u202f!\n',
 '\tCourez\u202f!\n',
 '\tPrenez vos jambes à vos cous !\n',
 '\tFile !\n',
 '\tFilez !\n',
 '\tCours !\n',
 '\tFuyez !\n',
 '\tFuyons !\n',
 '\tQui ?\n',
 '\tÇa alors\u202f!\n',
 '\tWaouh\xa0!\n',
 '\tWah\xa0!\n',
 '\tÀ terre\xa0!\n',
 '\tBaisse-toi\xa0!\n',
 '\tBaissez-vous\xa0!\n',
 '\tAu feu !\n',
 "\tÀ l'aide\u202f!\n",
 '\tCache-toi.\n',
 '\tCachez-vous.\n',
 '\tSaute.\n',
 '\tSaute.\n',
 '\tÇa suffit\u202f!\n',
 '\tStop\u202f!\n',
 '\tArrête-toi !\n',
 '\tAttends !\n',
 '\tAttendez !\n',
 '\tAttendez.\n',
 '\tAttends !\n',
 '\tAttendez !\n',
 '\tAttends.\n',
 '\tAttendez.\n',
 '\tCommencez.\n',
 '\tCommence.\n',
 '\tPoursuis.\n',
 '\tContinuez.\n',
 '\tPoursuivez.\n',
 '\tBonjour !\n',
 '\tSalut !\n',
 '\t

In [12]:
input_texts

['Go.',
 'Go.',
 'Go.',
 'Go.',
 'Hi.',
 'Hi.',
 'Run!',
 'Run!',
 'Run!',
 'Run!',
 'Run!',
 'Run!',
 'Run!',
 'Run!',
 'Run.',
 'Run.',
 'Run.',
 'Run.',
 'Run.',
 'Run.',
 'Run.',
 'Run.',
 'Who?',
 'Wow!',
 'Wow!',
 'Wow!',
 'Duck!',
 'Duck!',
 'Duck!',
 'Fire!',
 'Help!',
 'Hide.',
 'Hide.',
 'Jump!',
 'Jump.',
 'Stop!',
 'Stop!',
 'Stop!',
 'Wait!',
 'Wait!',
 'Wait!',
 'Wait.',
 'Wait.',
 'Wait.',
 'Wait.',
 'Begin.',
 'Begin.',
 'Go on.',
 'Go on.',
 'Go on.',
 'Hello!',
 'Hello!',
 'Hello.',
 'Hello.',
 'Hello.',
 'Hello.',
 'I see.',
 'I see.',
 'I try.',
 'I won!',
 'I won!',
 'I won.',
 'Oh no!',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Smile.',
 'Smile.',
 'Smile.',
 'Sorry?',
 'Attack!',
 'Attack!',
 'Attack!',
 'Attack!',
 'Buy it.',
 'Buy it.',
 'Buy it.',
 'Buy it.',
 'Cheers!',
 'Cheers!',
 'Cheers!',
 'Cheers!',
 'Eat it.',
 'Eat it.',
 'Exhale.',
 'Get up.',
 'Get up.',
 'Ge

In [13]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))

num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [14]:
max_encoder_seq_length

14

In [15]:
max_decoder_seq_length

59

In [16]:
print("Number of Unique Input Tokens: ", num_encoder_tokens)
print("Number of Unique Output Tokens: ", num_decoder_tokens)
print("Max Sequence Length for Input:", max_encoder_seq_length)
print("Max Sequence Length for Output:", max_decoder_seq_length)

Number of Unique Input Tokens:  70
Number of Unique Output Tokens:  93
Max Sequence Length for Input: 14
Max Sequence Length for Output: 59


 # Integer Encoding

In [17]:
input_token_index = dict(
    [(char,i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char,i) for i, char in enumerate(target_characters)])

In [18]:
input_token_index

{' ': 0,
 '!': 1,
 '"': 2,
 '$': 3,
 '%': 4,
 '&': 5,
 "'": 6,
 ',': 7,
 '-': 8,
 '.': 9,
 '0': 10,
 '1': 11,
 '2': 12,
 '3': 13,
 '5': 14,
 '7': 15,
 '8': 16,
 '9': 17,
 ':': 18,
 '?': 19,
 'A': 20,
 'B': 21,
 'C': 22,
 'D': 23,
 'E': 24,
 'F': 25,
 'G': 26,
 'H': 27,
 'I': 28,
 'J': 29,
 'K': 30,
 'L': 31,
 'M': 32,
 'N': 33,
 'O': 34,
 'P': 35,
 'Q': 36,
 'R': 37,
 'S': 38,
 'T': 39,
 'U': 40,
 'V': 41,
 'W': 42,
 'Y': 43,
 'a': 44,
 'b': 45,
 'c': 46,
 'd': 47,
 'e': 48,
 'f': 49,
 'g': 50,
 'h': 51,
 'i': 52,
 'j': 53,
 'k': 54,
 'l': 55,
 'm': 56,
 'n': 57,
 'o': 58,
 'p': 59,
 'q': 60,
 'r': 61,
 's': 62,
 't': 63,
 'u': 64,
 'v': 65,
 'w': 66,
 'x': 67,
 'y': 68,
 'z': 69}

In [19]:
target_token_index

{'\t': 0,
 '\n': 1,
 ' ': 2,
 '!': 3,
 '%': 4,
 '&': 5,
 "'": 6,
 '(': 7,
 ')': 8,
 ',': 9,
 '-': 10,
 '.': 11,
 '0': 12,
 '1': 13,
 '2': 14,
 '3': 15,
 '5': 16,
 '8': 17,
 '9': 18,
 ':': 19,
 '?': 20,
 'A': 21,
 'B': 22,
 'C': 23,
 'D': 24,
 'E': 25,
 'F': 26,
 'G': 27,
 'H': 28,
 'I': 29,
 'J': 30,
 'K': 31,
 'L': 32,
 'M': 33,
 'N': 34,
 'O': 35,
 'P': 36,
 'Q': 37,
 'R': 38,
 'S': 39,
 'T': 40,
 'U': 41,
 'V': 42,
 'W': 43,
 'Y': 44,
 'a': 45,
 'b': 46,
 'c': 47,
 'd': 48,
 'e': 49,
 'f': 50,
 'g': 51,
 'h': 52,
 'i': 53,
 'j': 54,
 'k': 55,
 'l': 56,
 'm': 57,
 'n': 58,
 'o': 59,
 'p': 60,
 'q': 61,
 'r': 62,
 's': 63,
 't': 64,
 'u': 65,
 'v': 66,
 'w': 67,
 'x': 68,
 'y': 69,
 'z': 70,
 '\xa0': 71,
 '«': 72,
 '»': 73,
 'À': 74,
 'Ç': 75,
 'É': 76,
 'Ê': 77,
 'à': 78,
 'â': 79,
 'ç': 80,
 'è': 81,
 'é': 82,
 'ê': 83,
 'î': 84,
 'ï': 85,
 'ô': 86,
 'ù': 87,
 'û': 88,
 'œ': 89,
 '\u2009': 90,
 '’': 91,
 '\u202f': 92}

# For One Hot Encoding

In [28]:
encoder_input_data = np.zeros(
    (len(input_texts),max_encoder_seq_length,num_encoder_tokens),dtype='float32')

decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),dtype='float32')

decoder_target_data = np.zeros(
    (len(input_texts),max_decoder_seq_length, num_decoder_tokens),dtype='float32')

In [29]:
for i, (input_text, target_text) in enumerate(zip(input_texts,target_texts)):
  for t, char in enumerate(input_text):
    encoder_input_data[i, t, input_token_index[char]] = 1.
  encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
  for t, char in enumerate(target_text):
    # decoder_target_data is ahead of decoder_input_data by one timmestamp
    decoder_input_data[i, t, target_token_index[char]] = 1.
    if t > 0:
      # decoder_taget_data will be ahead by one time stapm
      # and will not include the start charater
      decoder_taget_data[i, t - 1,target_token_index[char]] = 1.

  decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
  decoder_target_data[i, t:, target_token_index[' ']]  = 1.

In [30]:
decoder_target_data

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0.

In [31]:
decoder_input_data

array([[[1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.]],

       [[1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.]],

       [[1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.]],

       ...,

       [[1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0.

In [32]:
encoder_input_data

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [33]:
# Define an input sequence  and process it
encoder_inputs = Input(shape=(None,num_encoder_tokens))
encoder = LSTM (latent_dimension,return_state=True)
encoder_outputs, state_h,state_c = encoder(encoder_inputs)
# We Discard 'encoder_outputs' and only keep the states
encoder_states = [state_h, state_c]

In [35]:
# Set up the Decoder , using 'encoder_states' as initial state
decoder_inputs = Input(shape=(None,num_decoder_tokens))
# We set up our Decoder to Return Full Output Sequence
# and to Return Internal States as well . We don't use the
# Return Stattes in Training Model , But We will use them in infrerence
decoder_lstm = LSTM(latent_dimension,return_sequences=True, return_state=True)
decoder_outputs,_, _ = decoder_lstm(decoder_inputs,initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens,activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [36]:
# Define the Model That will turn
# 'encoder_input_data' & 'decoder_input_data' into 'decoder_target_data'
model = Model([encoder_inputs,decoder_inputs], decoder_outputs)

In [38]:
 # Run Training
 model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])

 model.fit([encoder_input_data,decoder_input_data],decoder_taget_data,
          batch_size=batch_s,
          epochs=epochs,
          validation_split=0.2)

Epoch 1/100
125/125 [==============================] - 6s 20ms/step - loss: 1.2176 - accuracy: 0.7322 - val_loss: 1.1328 - val_accuracy: 0.7131
Epoch 2/100
125/125 [==============================] - 1s 10ms/step - loss: 0.9427 - accuracy: 0.7472 - val_loss: 0.9734 - val_accuracy: 0.7247
Epoch 3/100
125/125 [==============================] - 1s 10ms/step - loss: 0.8550 - accuracy: 0.7635 - val_loss: 0.9093 - val_accuracy: 0.7472
Epoch 4/100
125/125 [==============================] - 1s 10ms/step - loss: 0.7708 - accuracy: 0.7862 - val_loss: 0.8078 - val_accuracy: 0.7680
Epoch 5/100
125/125 [==============================] - 1s 10ms/step - loss: 0.6846 - accuracy: 0.8040 - val_loss: 0.7537 - val_accuracy: 0.7781
Epoch 6/100
125/125 [==============================] - 1s 11ms/step - loss: 0.6379 - accuracy: 0.8147 - val_loss: 0.6929 - val_accuracy: 0.7991
Epoch 7/100
125/125 [==============================] - 1s 10ms/step - loss: 0.6066 - accuracy: 0.8225 - val_loss: 0.6650 - val_accuracy:

# Prediction

In [39]:
# Next: Inference mode(sampling).
# Here's The Drill:
# 1) Encode input and retrieve initinal state
# 2) Run one step of Decoder with this initial state
# And a " Start of Sequence " token as Target
# Output will be the next Target Token
# 3) Repeat with the current target token and current states


# Define Sampling Models
encoder_model = Model(encoder_inputs,encoder_states)

decoder_state_input_h = Input(shape=(latent_dimension,))
decoder_state_input_c = Input(shape=(latent_dimension,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs,state_h,state_c = decoder_lstm(decoder_inputs,initial_state=decoder_states_inputs)
decoder_states = [state_h,state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [40]:
# Reverse- lookup token index to decode sequences back to
# Somthing Readable.

revers_input_char_index = dict(
    (i,char) for char , i in input_token_index.items())

revers_target_char_index = dict(
    (i,char) for char, i in target_token_index.items())

In [41]:
revers_input_char_index

{0: ' ',
 1: '!',
 2: '"',
 3: '$',
 4: '%',
 5: '&',
 6: "'",
 7: ',',
 8: '-',
 9: '.',
 10: '0',
 11: '1',
 12: '2',
 13: '3',
 14: '5',
 15: '7',
 16: '8',
 17: '9',
 18: ':',
 19: '?',
 20: 'A',
 21: 'B',
 22: 'C',
 23: 'D',
 24: 'E',
 25: 'F',
 26: 'G',
 27: 'H',
 28: 'I',
 29: 'J',
 30: 'K',
 31: 'L',
 32: 'M',
 33: 'N',
 34: 'O',
 35: 'P',
 36: 'Q',
 37: 'R',
 38: 'S',
 39: 'T',
 40: 'U',
 41: 'V',
 42: 'W',
 43: 'Y',
 44: 'a',
 45: 'b',
 46: 'c',
 47: 'd',
 48: 'e',
 49: 'f',
 50: 'g',
 51: 'h',
 52: 'i',
 53: 'j',
 54: 'k',
 55: 'l',
 56: 'm',
 57: 'n',
 58: 'o',
 59: 'p',
 60: 'q',
 61: 'r',
 62: 's',
 63: 't',
 64: 'u',
 65: 'v',
 66: 'w',
 67: 'x',
 68: 'y',
 69: 'z'}

In [42]:
revers_target_char_index

{0: '\t',
 1: '\n',
 2: ' ',
 3: '!',
 4: '%',
 5: '&',
 6: "'",
 7: '(',
 8: ')',
 9: ',',
 10: '-',
 11: '.',
 12: '0',
 13: '1',
 14: '2',
 15: '3',
 16: '5',
 17: '8',
 18: '9',
 19: ':',
 20: '?',
 21: 'A',
 22: 'B',
 23: 'C',
 24: 'D',
 25: 'E',
 26: 'F',
 27: 'G',
 28: 'H',
 29: 'I',
 30: 'J',
 31: 'K',
 32: 'L',
 33: 'M',
 34: 'N',
 35: 'O',
 36: 'P',
 37: 'Q',
 38: 'R',
 39: 'S',
 40: 'T',
 41: 'U',
 42: 'V',
 43: 'W',
 44: 'Y',
 45: 'a',
 46: 'b',
 47: 'c',
 48: 'd',
 49: 'e',
 50: 'f',
 51: 'g',
 52: 'h',
 53: 'i',
 54: 'j',
 55: 'k',
 56: 'l',
 57: 'm',
 58: 'n',
 59: 'o',
 60: 'p',
 61: 'q',
 62: 'r',
 63: 's',
 64: 't',
 65: 'u',
 66: 'v',
 67: 'w',
 68: 'x',
 69: 'y',
 70: 'z',
 71: '\xa0',
 72: '«',
 73: '»',
 74: 'À',
 75: 'Ç',
 76: 'É',
 77: 'Ê',
 78: 'à',
 79: 'â',
 80: 'ç',
 81: 'è',
 82: 'é',
 83: 'ê',
 84: 'î',
 85: 'ï',
 86: 'ô',
 87: 'ù',
 88: 'û',
 89: 'œ',
 90: '\u2009',
 91: '’',
 92: '\u202f'}

In [56]:
def decode_sequence(input_seq):
  # Encode the input as state vectors
  states_value = encoder_model.predict(input_seq)

  # Generate Empty Target Sequence of Length 1.
  target_seq = np.zeros((1,1,num_decoder_tokens))
  # Populate the First Charecter of Target Sequnce with the Start Character.
  target_seq[0,0,target_token_index['\t']] = 1.

  # Sampling Loop for a batch of Sequences
  # to simplify , Here we assume a batch of size 1
  stop_condition = False
  decoded_sentence = ''
  while not stop_condition:
    output_tokens, h, c = decoder_model.predict([target_seq] + states_value)


    # Sample a Token

    sampled_token_index =np.argmax(output_tokens[0,-1,:])
    sampled_char = revers_target_char_index[sampled_token_index]
    decoded_sentence += sampled_char


    # Exit codition : eitheer hit max length or find stop character.
    if(sampled_char =='\n' or
       len(decoded_sentence) > max_decoder_seq_length):
      stop_condition=True



    # Update the target sequence (of length 1).
    target_seq = np.zeros((1,1,num_decoder_tokens))
    target_seq[0,0,sampled_token_index] = 1.


    # Update  States
    states_value = [h,c]

  return decoded_sentence


In [57]:
for seq_index in range(100):
  # Take one Sequence (part of the Training set) for trying out decode
  input_seq = encoder_input_data[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)
  print("-")
  print("Input Sentence :" ,input_texts[seq_index])
  print("Decoded Sentence : ",decoded_sentence)

1/1 [==============================] - 0s 18ms/step
-
Input Sentence : Go.
Decoded Sentence :  Au disous !

1/1 [==============================] - 0s 29ms/step
-
Input Sentence : Go.
Decoded Sentence :  Au disous !

1/1 [==============================] - 0s 28ms/step
-
Input Sentence : Go.
Decoded Sentence :  Au disous !

1/1 [==============================] - 0s 18ms/step
-
Input Sentence : Go.
Decoded Sentence :  Au disous !

1/1 [==============================] - 0s 17ms/step
-
Input Sentence : Hi.
Decoded Sentence :  Salut !

1/1 [==============================] - 0s 18ms/step
-
Input Sentence : Hi.
Decoded Sentence :  Salut !

1/1 [==============================] - 0s 21ms/step
-
Input Sentence : Run!
Decoded Sentence :  File !

1/1 [==============================] - 0s 18ms/step
-
Input Sentence : Run!
Decoded Sentence :  File !

1/1 [==============================] - 0s 18ms/step
-
Input Sentence : Run!
Decoded Sentence :  File !

1/1 [==============================] - 0s 18ms/s